# Predicción de ventas para Corporación Favorita

## Tabla de contenidos
[A. Definición del objetivo del problema](#a-definición-del-objetivo-del-problema)

[B. Recopilación de la información](#b-recopilación-de-la-información)
- [Días festivos](#días-festivos)
- [Petróleo](#petróleo)
- [Tiendas](#tiendas)
- [Train](#train)
- [Test](#test)
- [Transacciones](#transacciones)

[C. Overview y limpieza de la información](#c-overview-y-limpieza-de-la-información)
- [Días festivos](#limpieza-y-overview-días-festivos)
- [Petróleo](#limpieza-y-overview-petróleo)
- [Tiendas](#overview-tiendas)
- [Train](#overview-train)
- [Transacciones](#overview-transacciones)

[D. Visualización y relación entre features](#d-visualización-y-relación-entre-features)

[E. Análisis estadístico](#e-análisis-estadístico)

[F. Feature engineering](#f-feature-engineering)

[G. Tareas previas al modelo](#g-tareas-previas-al-modelo)

[H. Modelado](#h-modelado)

[I. Entrega de resultados](#i-entrega-de-resultado)


## A. Definición del objetivo del problema
Se busca obtener un modelo para predecir las ventas de Corporación Favorita, un supermercado ecuatoriano. Se busca que el modelo prediga con certeza las unidades de ventas para el supermercado con respecto al tiempo. 

## B. Recopilación de la información


### Días festivos
Los días festivos pueden proveer más tiempo para ir al supermercado o propiciar eventos familiares o de amigos que motiven a las personas a realizar más compras que no harían normalmente. 
| Columna | Descripción |
| --------|-------------|
| date | Fecha del día festivo |
| type | tipo de día festivo, si fue *Transferred* se celebró ese día cuando el día oficial es otro, si fue *Bridge* se agregó a otro |
| locale | A que nivel se celebró el día festivo (local, regional, nacional) |
| locale_name | en que localidad se celebró |
| description | descripción del día festivo |
| transfer | indica si el festivo se celebró en otra fecha, si el valor es verdadero la fecha en la que se celebró se indica con la columna **type** |

### Petróleo
Al ser un país dependiente del petróleo, el precio de este puede influir en las ventas del supermercado. 
| Columna | Descripción |
| --------|-------------|
| date | Fecha de la cotización del precio |
| dcoilwtico | precio del petroleo en la fecha |

### Tiendas
Información sobre cada tienda. 
| Columna | Descripción |
| --------|-------------|
| city | Ciudad |
| state | Estado |
| type | Tipo de tienda |
| cluster | agrupamiento de tiendas similares |

### Train
Datos de entrenamiento del modelo, se deberá usar la columna de **sales** como variable a predecir. 
| Columna | Descripción |
| --------|-------------|
| id | identificador único de la venta |
| date | Fecha de cada venta individual |
| store_nbr | tienda en la que se realizó la venta |
| family | familia de producto de la que se hizo la venta |
| sales | unidades vendidas de la familia de productos | 
| onpromotion | cantidad de artículos de la familia de productos que estaba en promoción | 

### Test
Datos para probar el modelo, igual que train menos la columna de **sales** ya que es la variable a predecir. 
| Columna | Descripción |
| --------|-------------|
| id | identificador único de la venta |
| date | Fecha de cada venta individual |
| store_nbr | tienda en la que se realizó la venta |
| family | familia de producto de la que se hizo la venta |
| onpromotion | cantidad de artículos de la familia de productos que estaba en promoción | 


### Transacciones

Datos de las transaccciones totales para cada tienda.   
| Columna | Descripción |
| --------|-------------|
| id | identificador único de la venta |
| date | Fecha de cada venta individual |
| store_nbr | tienda en la que se realizó la venta |
| family | familia de producto de la que se hizo la venta |
| onpromotion | cantidad de artículos de la familia de productos que estaba en promoción | 

## C. Overview y limpieza de la información

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

Debido a que la mayoría de las variables tienen los 100% completos, solamente se hará una ligera limpieza a Días Festivos y a Petróleo. Todas las variables se modificarán para que ahora su índice sea la fecha y se tratarán como variables tipo fecha de pandas.

### Limpieza y overview Días festivos

In [11]:
holidays_df = pd.read_csv('./data/holidays_events.csv')
holidays_df.tail(10)

,date,type,locale,locale_name,description,transferred
340,2017-12-06,Holiday,Local,Quito,Fundacion de Quito,True
341,2017-12-08,Holiday,Local,Loja,Fundacion de Loja,False
342,2017-12-08,Transfer,Local,Quito,Traslado Fundacion de Quito,False
343,2017-12-21,Additional,National,Ecuador,Navidad-4,False
344,2017-12-22,Holiday,Local,Salinas,Cantonizacion de Salinas,False
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False
349,2017-12-26,Additional,National,Ecuador,Navidad+1,False


In [26]:
# Cambiamos el dataset para que tome como festivos en los días en los que se celebró.
holidays_df.loc[(holidays_df.transferred), 'type'] = 'None'
holidays_df.loc[(holidays_df.type == 'Transfer'), 'type'] = 'Holiday'
# holidays_df['date'] = pd.to_datetime(holidays_df['date'])
# holidays_df = holidays_df.set_index('date')
holidays_df.tail(10)


,type,locale,locale_name,description,transferred
date,,,,,
2017-12-06,None,Local,Quito,Fundacion de Quito,True
2017-12-08,Holiday,Local,Loja,Fundacion de Loja,False
2017-12-08,Holiday,Local,Quito,Traslado Fundacion de Quito,False
2017-12-21,Additional,National,Ecuador,Navidad-4,False
2017-12-22,Holiday,Local,Salinas,Cantonizacion de Salinas,False
2017-12-22,Additional,National,Ecuador,Navidad-3,False
2017-12-23,Additional,National,Ecuador,Navidad-2,False
2017-12-24,Additional,National,Ecuador,Navidad-1,False
2017-12-25,Holiday,National,Ecuador,Navidad,False


In [13]:
holidays_df.shape

(350, 6)

### Limpieza y overview Petróleo

In [20]:
oil_prices_df = pd.read_csv('./data/oil.csv')
oil_prices_df = oil_prices_df.dropna()
print(oil_prices_df.shape)
oil_prices_df.head()

(1175, 2)


,date,dcoilwtico
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21


### Overview Tiendas

In [19]:
stores_df = pd.read_csv('./data/stores.csv')
print(stores_df.shape)
stores_df.head()

(54, 5)


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### Overview Train

In [27]:
train_df = pd.read_csv('./data/train.csv')
print(train_df.shape)
train_df['date'] = pd.to_datetime(train_df['date'])
train_df = train_df.set_index('date')
train_df[250000:250010]

(3000888, 6)


,id,store_nbr,family,sales,onpromotion
date,,,,,
2013-05-21,250000,23,PERSONAL CARE,93.000000,0
2013-05-21,250001,23,PET SUPPLIES,0.000000,0
2013-05-21,250002,23,PLAYERS AND ELECTRONICS,0.000000,0
2013-05-21,250003,23,POULTRY,125.645004,0
2013-05-21,250004,23,PREPARED FOODS,24.000000,0
2013-05-21,250005,23,PRODUCE,2.000000,0
2013-05-21,250006,23,SCHOOL AND OFFICE SUPPLIES,0.000000,0
2013-05-21,250007,23,SEAFOOD,12.900000,0
2013-05-21,250008,24,AUTOMOTIVE,9.000000,0


### Overview Transacciones

In [22]:
transactions_df = pd.read_csv('./data/transactions.csv')
print(transactions_df.shape)
transactions_df.head()

(83488, 3)


,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


## D. Visualización y relación entre features

## E. Análisis estadístico

## F. Feature engineering

## G. Tareas previas al modelo

## H. Modelado

## I. Entrega de resultado